In [ ]:
from mwasis import *  # noqa: F403
@tf.function(jit_compile=True)
def f(x):
    return x + 1


redirect = FDRedirector(STDERR)
redirect.start()
f(tf.constant(1))
redirect.stop();

## Wprowadzenie

- Jak poprawić model analityczny gdy wiem na przykładach ile jest niedokładny?
- Jak zbudować model gdy mamy tylko obserwacje?
- Probabilistyczny opis dopasowywania krzywych do punktów.
- Połączenie aspektów probabilistycznych i deterministycznych algorytmów.

## Modelowanie probabilistyczne

### Konstrukcja modelu

- Konstruujemy model tak jak symulator
- Obserwacje to jedne z możliwych wyników symulacji
- Uwzględniamy zależności pomiędzy zmiennymi!

### Wnioskowanie

- Jakie były parametry symulatora, który wygenerował to co obserwujemy?
- MLE
- Bayes (Nieznane parametry opisane rozkładami a priori)

## Modele hierarchiczne

Co jeżeli mamy grupy danych, lub inne skorelowane obserwacje?

- Różne grupy mogą mieć różne rozkłądy a priori
    - Hiper a priori - rozkład hiperparametrów
- czynniki zależne od grupy
- deterministyczne zależności

### Modele grafowe

- węzeł to zmienna losowa
- gałąź oznacza zależność

## Klasyczne modele hierarchiczne

![](bayesnet.pdf)


In [ ]:
from graphviz import Source

# Store DOT code as a string
dot_code = '''
digraph Optimizers {

    
    layout="dot";
    bgcolor="#FAFAFA"
    node [shape = circle, fontname = "FiraSans"];
    edge [fontname = "FiraSans"];
    0 [xlabel="stałe parametry",shape=point,width=0.2]
    hp [xlabel="Hiper a priori",label=""]
    0 -> hp
    hp-> "a priori 1"
    hp-> "a priori 2"
    hp-> "a priori 3"
    subgraph cluster_g1 {
        label="wiele obserwacji";
        o1 [label="obserwacje 1"];
    }
    subgraph cluster_g2 {
        label="wiele obserwacji";
        o2 [label="obserwacje 2"];
    }
    subgraph cluster_g3 {
        label="wiele obserwacji";
        o3[label="obserwacje 3"];
    }
    "a priori 1"->o1
    "a priori 2"->o2
    "a priori 3"->o3


}
'''
# Render the DOT code using graphviz's Source function
src = Source(dot_code)
src.render(view=False)  # Don't open an external viewer
src  # Display the graph in the notebook


## Reguła łańcuchowa

### Łaczny rozkład 

$${\displaystyle \mathrm {P} (X_{1},\ldots ,X_{n})=\mathrm {P} (X_{n}|X_{1},\ldots ,X_{n-1})\cdot \mathrm {P} (X_{1},\ldots ,X_{n-1})}$$


<!-- $${\displaystyle \mathrm {P} (X_{n},\ldots ,X_{1})=\mathrm {P} (X_{n}|X_{n-1},\ldots ,X_{1})\cdot \mathrm {P} (X_{n-1},\ldots ,X_{1})}$$
 -->

- Wzór Bayesa dla wielu zmiennych
- $\mathrm {P} (X_{n}|X_{1},\ldots ,X_{n-1})$ to najczęściej $\mathrm {P} (X_{n}|X_{n-1})$ lub $\mathrm {P} (X_{n}|X_{i},X_{j})$
- Umożliwia budowanie złożonych wielowymiarowych rozkładów z sekwencji rozkładów warunkowych
- Istotne w szeregach czasowych
- `tfp.distributions.JointDistribution*`



## Przykład 3 strumienie

- Dane: ruch $D_1,D_2,D_3$
- Szukane: $\lambda_1,\lambda_2,\lambda_3$

### model 

- $\lambda_i\sim \text{gamma}(\alpha, \beta),\quad i\in\{1,2\}$

- $P_b= f(\lambda_1+\lambda_2)$ (\alert{$M/M/1/b$})
- $\lambda_3 \sim \text{determ}( P_b\cdot(\lambda_1+\lambda_2))$ 
- $D_1\sim \text{poiss}(\lambda_1)$
- $D_2\sim \text{poiss}(\lambda_2)$
- $D_3\sim\text{poiss}(\lambda_3)$

<!-- # Bijekcje zmiennych losowych

- $Y=g(X)$

$$f_Y(y)=f_x(g^{-1}(y))\left|\frac{d }{d y}g^{-1}(y)\right|$$

- `tfp.distributions.TransformedDistribution`
- `tfp.bijectors` -->

## Probabilistyczne języki programowania

- Języki domenowe do wnioskowania statystycznego
- Zmienne losowe + zwykłe konstrukcje programistyczne (`for`,`while` itp)
- program piszemy tak, aby wygenerować możliwe dane
- Automatyczne wnioskowanie o rozkładach ukrytych zmiennych
- Często parze z programowanie różniczkowym (HMC, NUTS)



## Dopasowanie krzywych- problem

- Mamy $n$ obserwacji $X^{(i)}$ i $Y^{(i)}$
- Chcemy dopasowac krzywą $y=h(x)$
- Cel: szacowanie $y$ dla nowych obserwacji $x$

\begin{example}
$X$ to predykcja z modelu kolejkowego an $y$ to obserwowana wartość.

Model $y=f(x)$ to korekta uproszczonego modelu analitycznego
\end{example}

## Dopasowanie krzywych

Jeżeli model analityczny nie jest dokładny możemy dopasować ogólny model do obserwowanych danych.

<!-- - Obserwacje $X$ i $Y$, znając $x$ chcemy wiedzieć jaka będzie wartość $y$ -->

### Model

<!-- - Mając punkty $x_i$ i $y_i$ chcemy dopasować funkcje $y=h(x)$ do danych -->
- Model nie będzie dokładny
- Dla danego nowego $x$ nie znamy $y$ wiec reprezentujemy go jako zmienną losową o rozkładzie $Dist$
    - rozkład wyników $Y_i$ zależy of $x_i$ i parametrów modelu $\theta$ 
    - obserwowany wynik $y_i$ 

    $$ Y_i \sim Dist(x_i,\theta) $$ 
- $Y$ to wektor losowy parametryzowany przez $X$ i $\theta$

## Model liniowy

- $Y_i\sim Norm(\beta_0+\beta_1x_i,\sigma)$
- Równoważnie $y = \beta_0+\beta_1x + \epsilon, \quad \epsilon\sim Norm(0,\sigma)$
- Parametry $\theta=(\beta_0, \beta_1,\sigma)$

## Model liniowy wnioskowanie

### MLE

$$\ell=\sum_i-\frac{1}{2}\frac{(y_i-\beta_0 -\beta_1 x_i)^2}{\sigma^2}-n\log(\sigma)$$

### Bayes

- prior
- MCMC
- SVI

## Inne typy danych? 



- Binarne : rozkład Bernoulliego
- całkowite: rozkład Poissona
- Dodatnie: rozkład gamma
- $\ldots$

# Przykłady

## Regresja liniowa{.allowframebreaks}

```python
y_dist = tfp.distributions.Normal(
    loc=tfp.util.DeferredTensor(tf.Variable([0.0, 1.0]),
                                lambda beta: beta[0] + beta[1]*x,
                                shape=x.shape),
    scale=tfp.util.TransformedVariable([1.], bijector=tfp.bijectors.Softplus())
    )
```

In [ ]:
x = np.random.uniform(size=100)

y = 2*x +4 + np.random.normal(scale=0.4, size=x.shape)
sns.scatterplot(x=x,y=y);

In [ ]:
y_dist = tfp.distributions.Normal(
    loc=tfp.util.DeferredTensor(tf.Variable([0.0, 1.0]),
                                lambda beta: beta[0] + beta[1]*x,
                                shape=x.shape),
    scale=tfp.util.TransformedVariable([1.], bijector=tfp.bijectors.Softplus())
    )

y_dist.sample()

losses = tfp.math.minimize(lambda : -tf.reduce_sum(y_dist.log_prob(y)),
                           num_steps=2000,
                           optimizer=tf.optimizers.Adam(learning_rate=0.2))

In [ ]:
plt.plot(losses);

In [ ]:
sns.scatterplot(x=x,y=y)
y_hat = y_dist.mean().numpy()
y_sd = y_dist.stddev().numpy()

sns.lineplot(x=x, y=y_hat, label='model')

idx = np.argsort(x)
plt.fill_between(x[idx], y_hat[idx]-2*y_sd[idx], y_hat[idx]+2*y_sd[idx], alpha=0.2, label='95% ci')
plt.title(r'$y={:.3f}x+{:.3f} +N(0,{:.3f})$'.format(y_dist.variables[0][0].numpy(),y_dist.variables[0][1].numpy(),y_dist.stddev()[0].numpy()))
plt.legend();


## Klasyfikacja{.allowframebreaks}

```python
y_dist_bin = tfp.distributions.Bernoulli(logits=tfp.util.DeferredTensor(tf.Variable([-3.0, 8.0]),
                                lambda beta: beta[0] + beta[1]*x,
                                shape=x.shape),
    )
```

In [ ]:
y_dist_bin = tfp.distributions.Bernoulli(logits=tfp.util.DeferredTensor(tf.Variable([-3.0, 8.0]),
                                lambda beta: beta[0] + beta[1]*x,
                                shape=x.shape),
    )

_y=y_dist_bin.sample()

sns.scatterplot(x=x,y=_y, label='obserwacje')
sns.lineplot(x=x,y=y_dist_bin.mean(), label=r'$P(y=1)$')
plt.title(r'$\log\frac{p}{1-p}=8 x-3$');

## Regresja Poissonowska{.allowframebreaks}

```python
y_dist_bin = tfp.distributions.Poisson(log_rate=tfp.util.DeferredTensor(tf.Variable([-1.0, 3.]),
                                lambda beta: beta[0] + beta[1]*x,
                                shape=x.shape),
    )
```

In [ ]:
y_dist_bin = tfp.distributions.Poisson(log_rate=tfp.util.DeferredTensor(tf.Variable([-1.0, 3.]),
                                lambda beta: beta[0] + beta[1]*x,
                                shape=x.shape),
    )

_y=y_dist_bin.sample()

sns.scatterplot(x=x,y=_y, label='obserwacje')
sns.lineplot(x=x,y=y_dist_bin.mean(), label=r'$\lambda$')
plt.title("Regresja Poissonowska")
plt.xlabel('wydatki na reklamę')
plt.ylabel('liczba wejść na stronę')



## Podsumowanie

- Modele hierarchiczne uwzględniają złożoną strukturę zależności pomiędzy obserwowanymi danymi
- Dopasowanie krzywych to estymacja rozkładu wielowymiarowego
- Modele podobne do dopasowania krzywej można stosować dla nieciągłych danych


# {.standout}

Pytania